In [1]:
import os,sys,inspect
import json
import torch
currentdir = os.path.dirname(os.getcwd())
print(currentdir)
# parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(currentdir)
print("root Folder Absoloute path: ", root)


import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

import __main__

__main__.__file__="main.py"

/home/hfaghihi/Framework/fix/DomiKnowS/examples
root Folder Absoloute path:  /home/hfaghihi/Framework/fix/DomiKnowS


In [2]:
from regr.data.reader import RegrReader
import torch


class SudokuReader(RegrReader):
    def parse_file(self):
        base  = 3
        side  = base*base

        # pattern for a baseline valid solution
        def pattern(r,c): return (base*(r%base)+r//base+c)%side

        # randomize rows, columns and numbers (of valid base pattern)
        from random import sample
        def shuffle(s): return sample(s,len(s)) 
        rBase = range(base) 
        rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
        cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
        nums  = shuffle(range(1,base*base+1))

        # produce board using randomized baseline pattern
        board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
        squares = side*side
        empties = squares * 3//4
        for p in sample(range(squares),empties):
            board[p//side][p%side] = 0
        board = torch.tensor(board)
    
        return [{"board": board}]
    
    def getidval(self, item):
        return [1]
    def getwhole_sudokuval(self, item):
        return item['board']
            
    
    def getsizeval(self, item):
        return 9, 9
    

In [3]:
trainreader = SudokuReader("randn", type="raw")
item = next(iter(trainreader))
print(item)
indices = (item['whole_sudoku'] != 0).nonzero()
print(indices)
print(torch.where(item['whole_sudoku'] != 0))
print(item['whole_sudoku'][indices[:, 0], indices[:, 1]] )

{'id': [1], 'size': (9, 9), 'whole_sudoku': tensor([[0, 6, 1, 0, 8, 5, 0, 0, 3],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 8, 0, 0, 3, 0, 0, 0, 6],
        [0, 0, 3, 0, 0, 0, 0, 8, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 7, 0, 0],
        [8, 0, 0, 0, 0, 3, 0, 9, 7],
        [0, 0, 9, 0, 0, 0, 3, 0, 0],
        [3, 0, 0, 0, 0, 0, 8, 0, 0]])}
tensor([[0, 1],
        [0, 2],
        [0, 4],
        [0, 5],
        [0, 8],
        [1, 3],
        [2, 1],
        [2, 4],
        [2, 8],
        [3, 2],
        [3, 7],
        [4, 6],
        [5, 6],
        [6, 0],
        [6, 5],
        [6, 7],
        [6, 8],
        [7, 2],
        [7, 6],
        [8, 0],
        [8, 6]])
(tensor([0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 6, 6, 6, 7, 7, 8, 8]), tensor([1, 2, 4, 5, 8, 3, 1, 4, 8, 2, 7, 6, 6, 0, 5, 7, 8, 2, 6, 0, 6]))
tensor([6, 1, 8, 5, 3, 1, 8, 3, 6, 3, 8, 1, 7, 8, 3, 9, 7, 9, 3, 3, 8])


In [4]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL, fixedL, eqL
from regr.graph import EnumConcept


Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    sudoku = Concept("sodoku")
    
#     empty_entries = Concept(name="empty_entries")
#     fixed_entries = Concept(name="fixed_entries")
    
#     (sudoku_empty, sudoku_fixed) = sudoku.has_a(empty_entries, fixed_entries)
    
    empty_entry = Concept(name='empty_entry')
    (empty_rel, ) = sudoku.contains(empty_entry)
    
#     fixed_entry = Concept(name='fixed_entry')
#     (fixed_rel, ) = fixed_entries.contains(fixed_entry)
    
    same_row = Concept(name="same_row")
#     same_row_mixed = Concept(name="same_row_mixed")
    (same_row_arg1, same_row_arg2) = same_row.has_a(arg1=empty_entry, arg2=empty_entry)
#     (same_row_mixed_arg1, same_row_mixed_arg2) = same_row_mixed.has_a(arg1=empty_entry, arg2=fixed_entry)
    
    same_col = Concept(name="same_col")
#     same_col_mixed = Concept(name="same_col_mixed")
    (same_col_arg1, same_col_arg2) = same_col.has_a(col1=empty_entry, col2=empty_entry)
#     print(same_col_arg1)
#     (same_col_mixed_arg1, same_col_mixed_arg2) = same_col_mixed.has_a(arg1=empty_entry, arg2=fixed_entry)

    same_table = Concept(name="same_table")
    (same_table_arg1, same_table_arg2) = same_table.has_a(entry1=empty_entry, entry2=empty_entry)
    
    empty_entry_label = empty_entry(name="empty_entry_label", ConceptClass=EnumConcept, values=["v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9"])


    ### Constraints
    # entry = concept(name="entry")
    # entry["given"] = ReaderSensor(keyword="given")
    # entry_label= entry(name="label")
    # fixedL(entry_label("x", eqL(entry, "given", {True})))
    
#     fixedL(empty_entry_label("x", eqL(empty_entry, "fixed", {True})))
    
    #fixedL(empty_entry_label("x", path=('x', eqL(empty_entry, "fixed", {True}))))
    ifL(empty_entry, 
        exactL(getattr(empty_entry_label, f'v1'), getattr(empty_entry_label, f'v2'), getattr(empty_entry_label, f'v3'),
              getattr(empty_entry_label, f'v4'), getattr(empty_entry_label, f'v5'), getattr(empty_entry_label, f'v6'), getattr(empty_entry_label, f'v7'),
              getattr(empty_entry_label, f'v8'), getattr(empty_entry_label, f'v9'))
    )
    
    FIXED = True
    
    existsL_LCs = True
    atMostL_LCs = False
    
    SAME_ROW = existsL_LCs
    SAME_ROW_New = atMostL_LCs
    
    SAME_COLUMNE = existsL_LCs
    SAME_COLUMNE_New = atMostL_LCs
    
    SAME_TABLE = existsL_LCs
    SAME_TABLE_New = atMostL_LCs
    
    fixedL(empty_entry_label("x", eqL(empty_entry, "fixed", {True})), active = FIXED)
    
    #fixedL(empty_entry_label("x", path=('x', eqL(empty_entry, "fixed", {True}))))
    
    for val in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]:        
        ### No same number in the same row between empty entries and empty entries
        ifL(getattr(empty_entry_label, f'v{val}')('x'), 
            notL(
                existsL(
                    andL(
                        same_row('z', path=("x", same_row_arg1.reversed)), 
                        getattr(empty_entry_label, f'v{val}')('y', path=("z", same_row_arg2))
                ))
        ), active = SAME_ROW, name = "LC_SAME_ROW_ExistL_for_" + f'v{val}')
        
        ifL(getattr(empty_entry_label, f'v{val}')('x'), 
            atMostL(
                andL(
                    same_row('z', path=("x", same_row_arg1.reversed)), 
                    getattr(empty_entry_label, f'v{val}')('y', path=("z", same_row_arg2)
                )
            )
        ), active = SAME_ROW_New, name = "LC_SAME_ROW_atMostL_for_" + f'v{val}')
        
        ### No same number in the same column between empty entries and empty entries
        ifL(getattr(empty_entry_label, f'v{val}')('x'), 
            notL(
                existsL(
                    andL(
                        same_col('z', path=("x", same_col_arg1.reversed)), 
                        getattr(empty_entry_label, f'v{val}')('y', path=("z", same_col_arg2))
                ))
        ), active = SAME_COLUMNE, name = "LC_SAME_COLUMNE_existsL_for_" + f'v{val}')
        
        ifL(getattr(empty_entry_label, f'v{val}')('x'), 
            atMostL(
                andL(
                    same_col('z', path=("x", same_col_arg1.reversed)), 
                    getattr(empty_entry_label, f'v{val}')('y', path=("z", same_col_arg2)
                )
            )
        ), active = SAME_COLUMNE_New, name = "LC_SAME_COLUMNE_atMostL_for_" + f'v{val}')
        
        ### No same number in the same table between empty entries and empty entries
        ifL(getattr(empty_entry_label, f'v{val}')('x'), 
            notL(
                existsL(
                    andL(
                        same_table('z', path=("x", same_table_arg1.reversed)), 
                        getattr(empty_entry_label, f'v{val}')('y', path=("z", same_table_arg2))
                ))
        ), active = SAME_TABLE, name = "LC_SAME_TABLE_existsL_for_" + f'v{val}')
        
        ifL(getattr(empty_entry_label, f'v{val}')('x'), 
            atMostL(
                    andL(
                        same_table('z', path=("x", same_table_arg1.reversed)), 
                        getattr(empty_entry_label, f'v{val}')('y', path=("z", same_table_arg2)
                    )
            )
        ), active = SAME_TABLE_New, name = "LC_SAME_TABLE_atMostL_for_" + f'v{val}')


Log file for dataNode is in: /home/hfaghihi/Framework/fix/DomiKnowS/examples/sudoku/logs/datanode.log


main.py:26: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'empty_entry'), ('arg2', 'empty_entry')]) is used.
  
main.py:31: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('col1', 'empty_entry'), ('col2', 'empty_entry')]) is used.
  class SudokuReader(RegrReader):
main.py:36: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('entry1', 'empty_entry'), ('entry2', 'empty_entry')]) is used.
  # pattern for a baseline valid solution


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class Conv2dSame(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bias=True, padding_layer=torch.nn.ReflectionPad2d):
        """It only support square kernels and stride=1, dilation=1, groups=1."""
        super(Conv2dSame, self).__init__()
        ka = kernel_size // 2
        kb = ka - 1 if kernel_size % 2 == 0 else ka
        self.net = nn.Sequential(
            padding_layer((ka,kb,ka,kb)),
            nn.Conv2d(in_channels, out_channels, kernel_size, bias=bias),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
    def forward(self, x):
        return self.net(x)
    
class SudokuSolver(nn.Module):
    def __init__(self):
        super().__init__()
        self.W = torch.nn.Parameter(torch.rand((81,9)))
        
    def __call__(self, X):
        
        return self.W

In [6]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor, CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

class JointFunctionalReaderSensor(JointSensor, FunctionalReaderSensor):
    pass


# def getempties(*prev, data):
#     rows, cols = torch.where(data == 0)
#     rel = torch.ones(1, 1)
#     return [rows], [cols]
    
def getfixed(*prev, data):
    rows, cols = torch.where(data != 0)
    fix = torch.zeros(data.shape)
    vals = torch.ones(data.shape) * -1
    for i, j in zip(rows.detach().tolist(), cols.detach().tolist()):
        fix[i][j] = 1
        vals[i][j] = data[i][j]
        
        
    return fix.reshape(fix.shape[0]*fix.shape[1]), vals.reshape(vals.shape[0]*vals.shape[1])

def makeSoduko(*prev, data):
    num_rows = data[0]
    num_cols = data[1]
    rows = torch.arange(num_rows).unsqueeze(-1)
    rows = rows.repeat(1,num_cols).reshape(num_rows*num_cols)
    
    cols = torch.arange(num_rows)
    cols = cols.unsqueeze(0).repeat(num_rows, 1).reshape(num_rows*num_cols)
    
    rel = torch.ones(data[0]*data[1], 1)
    
    return rows, cols, rel

def getlabel(*prev, data):
    rows, cols = torch.where(data != 0)
    vals = torch.ones(data.shape) * -100
    for i, j in zip(rows.detach().tolist(), cols.detach().tolist()):
        vals[i][j] = data[i][j] - 1
    
#     print(vals.reshape(vals.shape[0]*vals.shape[1]))
#     print(vals.reshape(vals.shape[0]*vals.shape[1]).shape)
        
    return vals.reshape(vals.shape[0]*vals.shape[1])
    
    
def createSudoku(*prev, data):
    return [1]

sudoku['index'] = FunctionalReaderSensor(keyword='size', forward=createSudoku)
    
empty_entry['rows', 'cols', empty_rel] = JointFunctionalReaderSensor(sudoku['index'], keyword='size', forward=makeSoduko)
empty_entry['fixed', 'val'] = JointFunctionalReaderSensor('rows', 'cols', empty_rel, keyword='whole_sudoku', forward=getfixed)

empty_entry[empty_entry_label] = ModuleLearner('val', module=SudokuSolver())
empty_entry[empty_entry_label] = FunctionalReaderSensor(keyword='whole_sudoku', label=True, forward=getlabel)

def filter_col(*inputs, col1, col2):
    if col1.getAttribute('cols').item() == col2.getAttribute('cols').item() and col1.instanceID != col2.instanceID:
        return True
    return False
    
same_col[same_col_arg1.reversed, same_col_arg2.reversed] = CompositionCandidateSensor(
    empty_entry['cols'],
    relations=(same_col_arg1.reversed, same_col_arg2.reversed),
    forward=filter_col)

def filter_row(*inputs, arg1, arg2):
    if arg1.getAttribute('rows').item() == arg2.getAttribute('rows').item() and arg1.instanceID != arg2.instanceID:
        return True
    return False
    
same_row[same_row_arg1.reversed, same_row_arg2.reversed] = CompositionCandidateSensor(
    empty_entry['rows'],
    relations=(same_row_arg1.reversed, same_row_arg2.reversed),
    forward=filter_row)


def filter_table(*inputs, entry1, entry2):
    if entry1.instanceID != entry2.instanceID:
        if int(entry1.getAttribute('rows').item() / 3) == int(entry2.getAttribute('rows').item() / 3) and int(entry1.getAttribute('cols').item() / 3) == int(entry2.getAttribute('cols').item() / 3):
            return True
    return False
    
same_table[same_table_arg1.reversed, same_table_arg2.reversed] = CompositionCandidateSensor(
    empty_entry['rows'], empty_entry['cols'],
    relations=(same_table_arg1.reversed, same_table_arg2.reversed),
    forward=filter_table)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [9]:
from regr.program import POIProgram, SolverPOIProgram, IMLProgram, CallbackProgram
from regr.program.callbackprogram import ProgramStorageCallback
from regr.program.metric import MacroAverageTracker, PRF1Tracker, DatanodeCMMetric
from regr.program.lossprogram import SampleLossProgram
from regr.program.loss import NBCrossEntropyLoss, NBCrossEntropyIMLoss
from regr.program.model.pytorch import SolverModel

program1 = SolverPOIProgram(
        graph, poi=(sudoku, empty_entry, same_row, same_col, same_table), inferTypes=['local/argmax'],
        loss=MacroAverageTracker(NBCrossEntropyLoss()),
#         metric={
#             'argmax': PRF1Tracker(DatanodeCMMetric('local/argmax'))}
)

program = SampleLossProgram(
        graph, SolverModel,
        poi=(sudoku, empty_entry, same_row, same_col, same_table),
        inferTypes=['local/argmax'],
        # inferTypes=['ILP', 'local/argmax'],
#         metric={
#             'argmax': PRF1Tracker(DatanodeCMMetric('local/argmax'))},

        #metric={ 'softmax' : ValueTracker(prediction_softmax),
        #       'ILP': PRF1Tracker(DatanodeCMMetric()),
        #        'argmax': PRF1Tracker(DatanodeCMMetric('local/argmax'))
        #       },
#         loss=MacroAverageTracker(NBCrossEntropyLoss()),
        
        sample = True,
        sampleSize=300, 
        sampleGlobalLoss = False
        )

In [25]:
from IPython.utils import io

# with io.capture_output() as captured:
program1.train(trainreader, train_epoch_num=100,
              Optim=lambda param: torch.optim.SGD(param, lr=0.1), device='auto')

INFO:regr.program.program:Epoch: 1
INFO:regr.program.program:Training:
Epoch 1 Training: 100%|██████████| 1/1 [00:04<00:00,  4.40s/it]
INFO:regr.program.program: - loss:
INFO:regr.program.program:{'empty_entry_label': tensor(1.7064)}
INFO:regr.program.program:Epoch: 2
INFO:regr.program.program:Training:
Epoch 2 Training:   0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [9]:
data = None
for datanode in program1.populate(trainreader):
#     print(datanode)
#     datanode.inferILPResults(empty_entry_label, same_row, same_col, same_table, fun=None)
    data = datanode
    entries = datanode.getChildDataNodes(conceptName=empty_entry)
    table = torch.zeros(9, 9)
    for entry in entries:
#         print(entry.getAttribute('rows'))
#         print(entry.getAttribute('cols'))
#         print(entry.getAttribute('fixed'))
#         print(entry.getAttribute('val'))
#         print(entry.getAttribute(empty_entry_label, 'local/argmax'))
        t = entry.getAttribute(empty_entry_label, 'ILP')
#         print(t)
#         print(entry.getAttribute('fixed').item())
#         print((t == 1).nonzero(as_tuple=True)[0].item() + 1)
#         print((t == 1))
        assert len((t == 1).nonzero(as_tuple=True)) == 1
        predicted = (t == 1).nonzero(as_tuple=True)[0].item() + 1
        table[entry.getAttribute('rows').item()][entry.getAttribute('cols').item()] = predicted
        if entry.getAttribute('fixed').item() == 1:
            assert entry.getAttribute('val').item() == predicted
#         print("---")
    break

In [10]:
entries = data.getChildDataNodes(conceptName=empty_entry)
for entry in entries:
    same_cols = entry.getRelationLinks(relationName=same_col)
    assert len(same_cols) == 8
    for same_c in same_cols:
        arg1 = same_c.getRelationLinks('col1')[0]
        arg2 = same_c.getRelationLinks('col2')[0]
        assert entry.getAttribute('cols').item() == arg2.getAttribute('cols').item()
        
    same_rows = entry.getRelationLinks(relationName=same_row)
    assert len(same_rows) == 8
    for same_r in same_rows:
        arg1 = same_r.getRelationLinks('arg1')[0]
        arg2 = same_r.getRelationLinks('arg2')[0]
        assert entry.getAttribute('rows').item() == arg2.getAttribute('rows').item()
#     print("------")
    

In [11]:
table

tensor([[5., 2., 4., 9., 8., 7., 1., 3., 6.],
        [8., 6., 3., 1., 5., 2., 9., 7., 4.],
        [1., 7., 9., 3., 6., 4., 5., 8., 2.],
        [2., 9., 7., 8., 4., 5., 6., 1., 3.],
        [3., 1., 6., 7., 2., 9., 4., 5., 8.],
        [4., 5., 8., 6., 3., 1., 2., 9., 7.],
        [9., 3., 2., 5., 7., 6., 8., 4., 1.],
        [7., 4., 5., 2., 1., 8., 3., 6., 9.],
        [6., 8., 1., 4., 9., 3., 7., 2., 5.]])

In [26]:
from IPython.utils import io
from regr.utils import setProductionLogMode
setProductionLogMode()

# with io.capture_output() as captured:
program.train(trainreader, train_epoch_num=150, c_warmup_iters=0, 
              Optim=lambda param: torch.optim.SGD(param, lr=0.1), device='auto')



INFO:regr.program.program:Epoch: 1
INFO:regr.program.program:Training:
Epoch 1 Training: 100%|██████████| 1/1 [00:09<00:00,  9.83s/it]
INFO:regr.program.program: - loss:
INFO:regr.program.program:None
INFO:regr.program.program: - Constraint loss:
INFO:regr.program.program:{'LC1': tensor(5.5993), 'LC10': tensor(39.1760), 'LC17': tensor(39.1433), 'LC24': tensor(39.5451), 'LC31': tensor(40.3391), 'LC38': tensor(39.9354), 'LC45': tensor(39.7444), 'LC52': tensor(43.3714), 'LC59': tensor(43.6981), 'LC66': tensor(43.5512), 'LC73': tensor(39.5709), 'LC80': tensor(40.2105), 'LC87': tensor(39.8340), 'LC94': tensor(43.6860), 'LC101': tensor(43.2489), 'LC108': tensor(43.9003), 'LC115': tensor(42.3305), 'LC122': tensor(42.6638), 'LC129': tensor(42.2624), 'LC136': tensor(43.9275), 'LC143': tensor(44.5062), 'LC150': tensor(43.8287), 'LC157': tensor(45.0451), 'LC164': tensor(45.2656), 'LC171': tensor(44.8230), 'LC178': tensor(42.3420), 'LC185': tensor(41.9413), 'LC192': tensor(42.1943)}
INFO:regr.prog

KeyboardInterrupt: 

In [36]:
print(program.model.loss.value())
print(program1.model.loss)

None
None


In [19]:
for datanode in program.populate(trainreader):
    print(datanode)
    table = torch.zeros(9, 9)
    
    entries = datanode.getChildDataNodes(conceptName=empty_entry)
    for entry in entries:
        print(entry.getAttribute('rows'))
        print(entry.getAttribute('cols'))
#         print(entry.getAttribute(empty_entry_label, 'local/argmax'))
        t = entry.getAttribute(empty_entry_label, 'local/argmax')
#         print((t == 1).nonzero(as_tuple=True)[0].item() + 1)
        predicted = (t == 1).nonzero(as_tuple=True)[0].item() + 1
#         print(predicted)
        table[entry.getAttribute('rows').item()][entry.getAttribute('cols').item()] = predicted
#         print(predicted)
        if entry.getAttribute('fixed').item() == 1:
#             pass
            assert entry.getAttribute('val').item() == predicted
        print("---")
    break

sodoku 0
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(3, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(4, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(5, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(6, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(7, device='cuda:0')
---
tensor(0, device='cuda:0')
tensor(8, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(3, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(4, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(5, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(6, device='cuda:0')
---
tensor(1, device='cuda:0')
tensor(7, device='cuda:0')
---
tenso

In [18]:
table

tensor([[9., 4., 9., 3., 3., 4., 9., 9., 2.],
        [2., 6., 3., 9., 8., 2., 9., 3., 1.],
        [3., 4., 6., 7., 7., 3., 1., 4., 6.],
        [2., 7., 3., 3., 1., 6., 2., 2., 2.],
        [5., 5., 2., 9., 8., 7., 1., 6., 5.],
        [5., 7., 5., 6., 8., 1., 9., 1., 6.],
        [4., 1., 7., 3., 8., 3., 2., 5., 8.],
        [3., 9., 4., 8., 6., 7., 1., 1., 8.],
        [4., 5., 8., 1., 2., 8., 6., 7., 3.]])